In [11]:
import csv

def txt_to_csv(input_file, output_file):
    with open(input_file, 'r') as txtfile, open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for line in txtfile:
            # Dividir la línea utilizando ":::" como separador
            parts = line.strip().split(":::")
            # Escribir las partes en el archivo CSV
            csv_writer.writerow(parts)

# Cambia 'input.txt' por el nombre de tu archivo de texto y 'output.csv' por el nombre que quieras para el archivo CSV de salida.
txt_to_csv('train_data.txt', 'train_data.csv')
txt_to_csv('test_data.txt', 'test_data.csv')
txt_to_csv('test_data_solution.txt', 'test_data_solution.csv')


In [6]:
import pandas as pd

df = pd.read_csv('train_data.csv', header=None)
# Eliminar primera columna
df = df.drop(columns=[0])
# Asignar nombres a las columnas: 'title', 'genre', 'plot'
df.columns = ['title', 'genre', 'plot']
df.head()

,title,genre,plot
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [7]:
df.shape

(54214, 3)

In [8]:
df.genre.value_counts()

genre
 drama           13613
 documentary     13096
 comedy           7447
 short            5073
 horror           2204
 thriller         1591
 action           1315
 western          1032
 reality-tv        884
 family            784
 adventure         775
 music             731
 romance           672
 sci-fi            647
 adult             590
 crime             505
 animation         498
 sport             432
 talk-show         391
 fantasy           323
 mystery           319
 musical           277
 biography         265
 history           243
 game-show         194
 news              181
 war               132
Name: count, dtype: int64